## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jan. 26th Feb. 2nd


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,January-31-2025,Condo/Co-op,11213 SW 88th St Unit B211,Miami,FL,33176.0,228000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/11213-SW-88th-...,MARMLS,A11736740,N,Y,25.68845,-80.374911


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [9]:
df2 = df

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-01-31    28
2025-01-27    26
2025-01-30    18
2025-01-28    11
2025-01-29    11
2025-02-02     2
2025-02-03     1
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
96,PAST SALE,2025-01-27,Condo/Co-op,18501 Collins Ave #3802,Sunny Isles Beach,FL,33160.0,5475000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11539301,N,Y,25.947720,-80.120228
6,PAST SALE,2025-01-28,Condo/Co-op,500 Alton Rd #4405,Miami Beach,FL,33139.0,5149910.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11678807,N,Y,25.775127,-80.141257
92,PAST SALE,2025-01-27,Condo/Co-op,9601 Collins Ave Ph 206,Bal Harbour,FL,33154.0,3650000.0,2.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11566310,N,Y,25.887706,-80.122180
13,PAST SALE,2025-01-29,Condo/Co-op,9601 Collins Ave #1608,Bal Harbour,FL,33154.0,3300000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11703368,N,Y,25.887706,-80.122180
84,PAST SALE,2025-02-02,Condo/Co-op,17121 Collins Ave #3304,Sunny Isles Beach,FL,33160.0,2850000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11623617,N,Y,25.934401,-80.121005
97,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #4201,Miami,FL,33131.0,2800000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/848-Brickell-K...,MARMLS,A11451983,N,Y,25.769433,-80.183317
77,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #3701,Miami,FL,33131.0,2525000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/848-Brickell-K...,MARMLS,A11633034,N,Y,25.769433,-80.183317
69,PAST SALE,2025-01-31,Condo/Co-op,10201 E Bay Harbor Dr #203,Bay Harbor Islands,FL,33154.0,2200000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11634043,N,Y,25.892681,-80.130898
74,PAST SALE,2025-01-30,Condo/Co-op,3400 SW 27th Ave #304,Miami,FL,33133.0,1750000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3400-SW-27th-A...,MARMLS,A11651850,N,Y,25.729664,-80.237811
71,PAST SALE,2025-02-03,Condo/Co-op,650 NE 32nd St #4001,Miami,FL,33137.0,1575000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/650-NE-32nd-St...,MARMLS,A11664569,N,Y,25.806902,-80.186249


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-01-27,Condo/Co-op,18501 Collins Ave #3802,Sunny Isles Beach,FL,33160.0,5475000.0,3.0,4.5,...,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniela Carvalho,"Jad Realty, LLC.",None,None,Mariana Shulga,"Mariana Shulga, LLC.",None,None
1,PAST SALE,2025-01-28,Condo/Co-op,500 Alton Rd #4405,Miami Beach,FL,33139.0,5149910.0,3.0,3.5,...,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Christopher Wands,Douglas Elliman,Anca Mirescu,Douglas Elliman,Christopher Wands,Douglas Elliman,Janet Hoyos,Douglas Elliman
2,PAST SALE,2025-01-27,Condo/Co-op,9601 Collins Ave Ph 206,Bal Harbour,FL,33154.0,3650000.0,2.0,3.5,...,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Karen Reiter,One Sotheby's International Realty,None,None,Joelle Oiknine,One Sotheby's International Realty,Marie Laure Miller,One Sotheby's International Realty
3,PAST SALE,2025-01-29,Condo/Co-op,9601 Collins Ave #1608,Bal Harbour,FL,33154.0,3300000.0,2.0,2.5,...,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Joelle Oiknine,One Sotheby's International Realty,Marie Laure Miller,One Sotheby's International Realty,Joelle Oiknine,One Sotheby's International Realty,None,None
4,PAST SALE,2025-02-02,Condo/Co-op,17121 Collins Ave #3304,Sunny Isles Beach,FL,33160.0,2850000.0,3.0,3.5,...,-80.121005,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,None,None,None,None,None,None,None,None
5,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #4201,Miami,FL,33131.0,2800000.0,3.0,2.5,...,-80.183317,https://www.redfin.com/FL/Miami/848-Brickell-K...,Daysi Morey,Berkshire Hathaway HomeServices Florida Realty,None,None,Maggie Joya,BHHS EWM Realty,None,None
6,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #3701,Miami,FL,33131.0,2525000.0,3.0,3.0,...,-80.183317,https://www.redfin.com/FL/Miami/848-Brickell-K...,Al Beteta,"Solid Group Realty, LLC",None,None,Katherina Santana,Keller Williams Capital Realty,None,None
7,PAST SALE,2025-01-31,Condo/Co-op,10201 E Bay Harbor Dr #203,Bay Harbor Islands,FL,33154.0,2200000.0,3.0,3.5,...,-80.130898,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Fernando Rodriguez,Fortune Christie's International Real Estate,None,None,Viktoriya Donisova Ho,"Compass Florida, LLC.",None,None
8,PAST SALE,2025-01-30,Condo/Co-op,3400 SW 27th Ave #304,Miami,FL,33133.0,1750000.0,2.0,2.5,...,-80.237811,https://www.redfin.com/FL/Miami/3400-SW-27th-A...,Maite Alvarez,"Compass Florida, LLC.",None,None,Mitzi Mitchell,Cervera Real Estate Inc.,None,None
9,PAST SALE,2025-02-03,Condo/Co-op,650 NE 32nd St #4001,Miami,FL,33137.0,1575000.0,4.0,4.5,...,-80.186249,https://www.redfin.com/FL/Miami/650-NE-32nd-St...,Adam Redolfi,Alure Capital,None,None,Adam Redolfi,Alure Capital,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

97


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$733,712


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$525


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$71,170,110


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
85
Input Previous Week Condo Average Sales Price:
1105012
Input Previous Week Condo Average PSF:
598
Input Previous Week Condo Sales total (ex: 198_000_000)
93_900_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-27,Condo/Co-op,18501 Collins Ave #3802,Sunny Isles Beach,FL,33160.0,5475000.0,3.0,4.5,NORTH BISCAYNE BEACH AMEN,3110.0,NaN,2020.0,NaN,1760.0,5260.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11539301,N,Y,25.94772,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniela Carvalho,"Jad Realty, LLC.",None,None,Mariana Shulga,"Mariana Shulga, LLC.",None,None,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-27,Condo/Co-op,18501 Collins Ave #3802,Sunny Isles Beach,FL,33160.0,5475000.0,3.0,4.5,NORTH BISCAYNE BEACH AMEN,3110.0,NaN,2020.0,NaN,1760.0,5260.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11539301,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniela Carvalho,"Jad Realty, LLC.",None,None,Mariana Shulga,"Mariana Shulga, LLC.",None,None,1,orange
1,PAST SALE,2025-01-28,Condo/Co-op,500 Alton Rd #4405,Miami Beach,FL,33139.0,5149910.0,3.0,3.5,AQUARIUM SITE AMD,1969.0,NaN,2023.0,NaN,2615.0,3524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11678807,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Christopher Wands,Douglas Elliman,Anca Mirescu,Douglas Elliman,Christopher Wands,Douglas Elliman,Janet Hoyos,Douglas Elliman,2,blue
2,PAST SALE,2025-01-27,Condo/Co-op,9601 Collins Ave Ph 206,Bal Harbour,FL,33154.0,3650000.0,2.0,3.5,MAJESTIC TOWER,2690.0,NaN,1998.0,NaN,1357.0,3413.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11566310,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Karen Reiter,One Sotheby's International Realty,None,None,Joelle Oiknine,One Sotheby's International Realty,Marie Laure Miller,One Sotheby's International Realty,3,blue
3,PAST SALE,2025-01-29,Condo/Co-op,9601 Collins Ave #1608,Bal Harbour,FL,33154.0,3300000.0,2.0,2.5,MAJESTIC TOWER,2250.0,NaN,1998.0,NaN,1467.0,2389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11703368,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Joelle Oiknine,One Sotheby's International Realty,Marie Laure Miller,One Sotheby's International Realty,Joelle Oiknine,One Sotheby's International Realty,None,None,4,blue
4,PAST SALE,2025-02-02,Condo/Co-op,17121 Collins Ave #3304,Sunny Isles Beach,FL,33160.0,2850000.0,3.0,3.5,JADE OCEAN CONDO,1933.0,NaN,2009.0,NaN,1474.0,2872.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11623617,N,Y,25.934401,-80.121005,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,None,None,None,None,None,None,None,None,5,blue
5,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #4201,Miami,FL,33131.0,2800000.0,3.0,2.5,THREE TEQUESTA POINT COND,2353.0,NaN,2001.0,NaN,1190.0,2338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/848-Brickell-K...,MARMLS,A11451983,N,Y,25.769433,-80.183317,https://www.redfin.com/FL/Miami/848-Brickell-K...,Daysi Morey,Berkshire Hathaway HomeServices Florida Realty,None,None,Maggie Joya,BHHS EWM Realty,None,None,6,blue
6,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #3701,Miami,FL,33131.0,2525000.0,3.0,3.0,THREE TEQUESTA POINT COND,2353.0,NaN,2001.0,NaN,1073.0,2338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/848-Brickell-K...,MARMLS,A11633034,N,Y,25.769433,-80.183317,https://www.redfin.com/FL/Miami/848-Brickell-K...,Al Beteta,"Solid Group Realty, LLC",None,None,Katherina Santana,Keller Williams Capital Realty,None,None,7,blue
7,PAST SALE,2025-01-31,Condo/Co-op,10201 E Bay Harbor Dr #203,Bay Harbor Islands,FL,33154.0,2200000.0,3.0,3.5,SERENO RESIDENCES CONDO,1660.0,NaN,2016.0,NaN,1325.0,2603.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11634043,N,Y,25.892681,-80.130898,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Fernando Rodriguez,Fortune Christie's International Real Estate,None,None,Viktoriya Donisova Ho,"Compass Florida, LLC.",None,None,8,blue
8,PAST SALE,2025-01-30,Condo/Co-op,3400 SW 27th Ave #304,

In [32]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [33]:
m.save('index.html')

## Data snagger

In [34]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [35]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-27,Condo/Co-op,18501 Collins Ave #3802,Sunny Isles Beach,FL,33160.0,5475000.0,3.0,4.5,NORTH BISCAYNE BEACH AMEN,3110.0,NaN,2020.0,NaN,1760.0,5260.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11539301,N,Y,25.94772,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniela Carvalho,"Jad Realty, LLC.",None,None,Mariana Shulga,"Mariana Shulga, LLC.",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_02032024


# CREATE TEMPLATE 

In [37]:
muni_set = set(df_top_ten['CITY'])

In [38]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [39]:
df_top_ten.reset_index(inplace=True,drop=True)

In [40]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [41]:
top_sale

'NORTH BISCAYNE BEACH AMEN at 18501 Collins Ave #3802 in Sunny Isles Beach'

In [42]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [43]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [44]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-27,Condo/Co-op,18501 Collins Ave #3802,Sunny Isles Beach,FL,33160.0,5475000.0,3.0,4.5,NORTH BISCAYNE BEACH AMEN,3110.0,NaN,2020.0,NaN,1760.0,5260.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11539301,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniela Carvalho,"Jad Realty, LLC.",None,None,Mariana Shulga,"Mariana Shulga, LLC.",None,None,1,orange,NORTH BISCAYNE BEACH AMEN at 18501 Collins Ave...
1,PAST SALE,2025-01-28,Condo/Co-op,500 Alton Rd #4405,Miami Beach,FL,33139.0,5149910.0,3.0,3.5,AQUARIUM SITE AMD,1969.0,NaN,2023.0,NaN,2615.0,3524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11678807,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Christopher Wands,Douglas Elliman,Anca Mirescu,Douglas Elliman,Christopher Wands,Douglas Elliman,Janet Hoyos,Douglas Elliman,2,blue,AQUARIUM SITE AMD at 500 Alton Rd #4405 in Mia...
2,PAST SALE,2025-01-27,Condo/Co-op,9601 Collins Ave Ph 206,Bal Harbour,FL,33154.0,3650000.0,2.0,3.5,MAJESTIC TOWER,2690.0,NaN,1998.0,NaN,1357.0,3413.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11566310,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Karen Reiter,One Sotheby's International Realty,None,None,Joelle Oiknine,One Sotheby's International Realty,Marie Laure Miller,One Sotheby's International Realty,3,blue,MAJESTIC TOWER at 9601 Collins Ave Ph 206 in B...
3,PAST SALE,2025-01-29,Condo/Co-op,9601 Collins Ave #1608,Bal Harbour,FL,33154.0,3300000.0,2.0,2.5,MAJESTIC TOWER,2250.0,NaN,1998.0,NaN,1467.0,2389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11703368,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Joelle Oiknine,One Sotheby's International Realty,Marie Laure Miller,One Sotheby's International Realty,Joelle Oiknine,One Sotheby's International Realty,None,None,4,blue,MAJESTIC TOWER at 9601 Collins Ave #1608 in Ba...
4,PAST SALE,2025-02-02,Condo/Co-op,17121 Collins Ave #3304,Sunny Isles Beach,FL,33160.0,2850000.0,3.0,3.5,JADE OCEAN CONDO,1933.0,NaN,2009.0,NaN,1474.0,2872.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11623617,N,Y,25.934401,-80.121005,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,None,None,None,None,None,None,None,None,5,blue,JADE OCEAN CONDO at 17121 Collins Ave #3304 in...
5,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #4201,Miami,FL,33131.0,2800000.0,3.0,2.5,THREE TEQUESTA POINT COND,2353.0,NaN,2001.0,NaN,1190.0,2338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/848-Brickell-K...,MARMLS,A11451983,N,Y,25.769433,-80.183317,https://www.redfin.com/FL/Miami/848-Brickell-K...,Daysi Morey,Berkshire Hathaway HomeServices Florida Realty,None,None,Maggie Joya,BHHS EWM Realty,None,None,6,blue,THREE TEQUESTA POINT COND at 848 Brickell Key ...
6,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #3701,Miami,FL,33131.0,2525000.0,3.0,3.0,THREE TEQUESTA POINT COND,2353.0,NaN,2001.0,NaN,1073.0,2338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/848-Brickell-K...,MARMLS,A11633034,N,Y,25.769433,-80.183317,https://www.redfin.com/FL/Miami/848-Brickell-K...,Al Beteta,"Solid Group Realty, LLC",None,None,Katherina Santana,Keller Williams Capital Realty,None,None,7,blue,THREE TEQUESTA POINT COND at 848 Brickell Key ...
7,PAST SALE,2025-01-31,Condo/Co-op,10201 E Bay Harbor Dr #203,Bay Harbor Islands,FL,33154.0,2200000.0,3.0,3.5,SERENO RESIDENCES CONDO,1660.0,NaN,2016.0,NaN,132

In [45]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: NORTH BISCAYNE BEACH AMEN closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,575,000 to $5,475,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Bal Harbour, Sunny Isles Beach, Miami, Bay Harbor Islands, Miami Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 97 condo sales totaling $71,170,110 million from Jan. 26th to Feb. 2nd. The previous week, brokers closed 85 condo sales totaling $93,900,000.

Last week’s units sold for an average of $733,712, lower than the $1,105,012 average price from th

In [46]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [47]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-27,Condo/Co-op,18501 Collins Ave #3802,Sunny Isles Beach,FL,33160.0,5475000.0,3.0,4.5,NORTH BISCAYNE BEACH AMEN,3110.0,NaN,2020.0,NaN,1760.0,5260.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11539301,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniela Carvalho,"Jad Realty, LLC.",None,None,Mariana Shulga,"Mariana Shulga, LLC.",None,None,1,orange,NORTH BISCAYNE BEACH AMEN at 18501 Collins Ave...
1,PAST SALE,2025-01-28,Condo/Co-op,500 Alton Rd #4405,Miami Beach,FL,33139.0,5149910.0,3.0,3.5,AQUARIUM SITE AMD,1969.0,NaN,2023.0,NaN,2615.0,3524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11678807,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Christopher Wands,Douglas Elliman,Anca Mirescu,Douglas Elliman,Christopher Wands,Douglas Elliman,Janet Hoyos,Douglas Elliman,2,blue,AQUARIUM SITE AMD at 500 Alton Rd #4405 in Mia...
2,PAST SALE,2025-01-27,Condo/Co-op,9601 Collins Ave Ph 206,Bal Harbour,FL,33154.0,3650000.0,2.0,3.5,MAJESTIC TOWER,2690.0,NaN,1998.0,NaN,1357.0,3413.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11566310,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Karen Reiter,One Sotheby's International Realty,None,None,Joelle Oiknine,One Sotheby's International Realty,Marie Laure Miller,One Sotheby's International Realty,3,blue,MAJESTIC TOWER at 9601 Collins Ave Ph 206 in B...
3,PAST SALE,2025-01-29,Condo/Co-op,9601 Collins Ave #1608,Bal Harbour,FL,33154.0,3300000.0,2.0,2.5,MAJESTIC TOWER,2250.0,NaN,1998.0,NaN,1467.0,2389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11703368,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Joelle Oiknine,One Sotheby's International Realty,Marie Laure Miller,One Sotheby's International Realty,Joelle Oiknine,One Sotheby's International Realty,None,None,4,blue,MAJESTIC TOWER at 9601 Collins Ave #1608 in Ba...
4,PAST SALE,2025-02-02,Condo/Co-op,17121 Collins Ave #3304,Sunny Isles Beach,FL,33160.0,2850000.0,3.0,3.5,JADE OCEAN CONDO,1933.0,NaN,2009.0,NaN,1474.0,2872.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11623617,N,Y,25.934401,-80.121005,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,None,None,None,None,None,None,None,None,5,blue,JADE OCEAN CONDO at 17121 Collins Ave #3304 in...
5,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #4201,Miami,FL,33131.0,2800000.0,3.0,2.5,THREE TEQUESTA POINT COND,2353.0,NaN,2001.0,NaN,1190.0,2338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/848-Brickell-K...,MARMLS,A11451983,N,Y,25.769433,-80.183317,https://www.redfin.com/FL/Miami/848-Brickell-K...,Daysi Morey,Berkshire Hathaway HomeServices Florida Realty,None,None,Maggie Joya,BHHS EWM Realty,None,None,6,blue,THREE TEQUESTA POINT COND at 848 Brickell Key ...
6,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #3701,Miami,FL,33131.0,2525000.0,3.0,3.0,THREE TEQUESTA POINT COND,2353.0,NaN,2001.0,NaN,1073.0,2338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/848-Brickell-K...,MARMLS,A11633034,N,Y,25.769433,-80.183317,https://www.redfin.com/FL/Miami/848-Brickell-K...,Al Beteta,"Solid Group Realty, LLC",None,None,Katherina Santana,Keller Williams Capital Realty,None,None,7,blue,THREE TEQUESTA POINT COND at 848 Brickell Key ...
7,PAST SALE,2025-01-31,Condo/Co-op,10201 E Bay Harbor Dr #203,Bay Harbor Islands,FL,33154.0,2200000.0,3.0,3.5,SERENO RESIDENCES CONDO,1660.0,NaN,2016.0,NaN,132

In [50]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: NORTH BISCAYNE BEACH AMEN closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,575,000 to $5,475,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Bal Harbour, Sunny Isles Beach, Miami, Bay Harbor Islands, Miami Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 97 condo sales totaling $71,170,110 million from Jan. 26th to Feb. 2nd. The previous week, brokers closed 85 condo sales totaling $93,900,000.

Last week’s units sold for an average of $733,712, lower than the $1,105,012 average price from th

In [58]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Sunny-Isles-Beach/18501-Collins-Ave-33160/unit-3802/home/175110950


In [59]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/500-Alton-Rd-33139/unit-4405/home/193222993


In [60]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/650-NE-32nd-St-33137/unit-4001/home/56745331


In [61]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/3400-SW-27th-Ave-33133/unit-304/home/43265618


In [62]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/500-Alton-Rd-33139/unit-4405/home/193222993


In [63]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/650-NE-32nd-St-33137/unit-4001/home/56745331


In [64]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-27,Condo/Co-op,18501 Collins Ave #3802,Sunny Isles Beach,FL,33160.0,5475000.0,3.0,4.5,NORTH BISCAYNE BEACH AMEN,3110.0,NaN,2020.0,NaN,1760.0,5260.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11539301,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniela Carvalho,"Jad Realty, LLC.",None,None,Mariana Shulga,"Mariana Shulga, LLC.",None,None,1,orange,NORTH BISCAYNE BEACH AMEN at 18501 Collins Ave...
1,PAST SALE,2025-01-28,Condo/Co-op,500 Alton Rd #4405,Miami Beach,FL,33139.0,5149910.0,3.0,3.5,AQUARIUM SITE AMD,1969.0,NaN,2023.0,NaN,2615.0,3524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11678807,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Christopher Wands,Douglas Elliman,Anca Mirescu,Douglas Elliman,Christopher Wands,Douglas Elliman,Janet Hoyos,Douglas Elliman,2,blue,AQUARIUM SITE AMD at 500 Alton Rd #4405 in Mia...
2,PAST SALE,2025-01-27,Condo/Co-op,9601 Collins Ave Ph 206,Bal Harbour,FL,33154.0,3650000.0,2.0,3.5,MAJESTIC TOWER,2690.0,NaN,1998.0,NaN,1357.0,3413.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11566310,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Karen Reiter,One Sotheby's International Realty,None,None,Joelle Oiknine,One Sotheby's International Realty,Marie Laure Miller,One Sotheby's International Realty,3,blue,MAJESTIC TOWER at 9601 Collins Ave Ph 206 in B...
3,PAST SALE,2025-01-29,Condo/Co-op,9601 Collins Ave #1608,Bal Harbour,FL,33154.0,3300000.0,2.0,2.5,MAJESTIC TOWER,2250.0,NaN,1998.0,NaN,1467.0,2389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11703368,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Joelle Oiknine,One Sotheby's International Realty,Marie Laure Miller,One Sotheby's International Realty,Joelle Oiknine,One Sotheby's International Realty,None,None,4,blue,MAJESTIC TOWER at 9601 Collins Ave #1608 in Ba...
4,PAST SALE,2025-02-02,Condo/Co-op,17121 Collins Ave #3304,Sunny Isles Beach,FL,33160.0,2850000.0,3.0,3.5,JADE OCEAN CONDO,1933.0,NaN,2009.0,NaN,1474.0,2872.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11623617,N,Y,25.934401,-80.121005,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,None,None,None,None,None,None,None,None,5,blue,JADE OCEAN CONDO at 17121 Collins Ave #3304 in...
5,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #4201,Miami,FL,33131.0,2800000.0,3.0,2.5,THREE TEQUESTA POINT COND,2353.0,NaN,2001.0,NaN,1190.0,2338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/848-Brickell-K...,MARMLS,A11451983,N,Y,25.769433,-80.183317,https://www.redfin.com/FL/Miami/848-Brickell-K...,Daysi Morey,Berkshire Hathaway HomeServices Florida Realty,None,None,Maggie Joya,BHHS EWM Realty,None,None,6,blue,THREE TEQUESTA POINT COND at 848 Brickell Key ...
6,PAST SALE,2025-01-31,Condo/Co-op,848 Brickell Key Dr #3701,Miami,FL,33131.0,2525000.0,3.0,3.0,THREE TEQUESTA POINT COND,2353.0,NaN,2001.0,NaN,1073.0,2338.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/848-Brickell-K...,MARMLS,A11633034,N,Y,25.769433,-80.183317,https://www.redfin.com/FL/Miami/848-Brickell-K...,Al Beteta,"Solid Group Realty, LLC",None,None,Katherina Santana,Keller Williams Capital Realty,None,None,7,blue,THREE TEQUESTA POINT COND at 848 Brickell Key ...
7,PAST SALE,2025-01-31,Condo/Co-op,10201 E Bay Harbor Dr #203,Bay Harbor Islands,FL,33154.0,2200000.0,3.0,3.5,SERENO RESIDENCES CONDO,1660.0,NaN,2016.0,NaN,132

## Time on Market Calculator

In [66]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 10, 21) ## List (Earlier) date
date2 = datetime(2025, 1, 28) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

99


## Clean CSV for Datawrapper Chart

In [67]:
chart_df = df_top_ten

In [68]:
chart_df = chart_df.fillna(" ")

In [69]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [70]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [71]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [72]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [73]:
chart_df['$/SQUARE FEET'].astype(float)

0    1760.0
1    2615.0
2    1357.0
3    1467.0
4    1474.0
5    1190.0
6    1073.0
7    1325.0
8    1144.0
9     841.0
Name: $/SQUARE FEET, dtype: float64

In [74]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [75]:
csv_date_string = today.strftime("%m_%d_%Y")

In [76]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [77]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-27,18501 Collins Ave #3802 in Sunny Isles Beach,"$5,475,000",3,4.5,"3,110",2020,"$1,760",MARMLS,A11539301,"Daniela Carvalho Jad Realty, LLC.","Mariana Shulga Mariana Shulga, LLC."
1,2025-01-28,500 Alton Rd #4405 in Miami Beach,"$5,149,910",3,3.5,"1,969",2023,"$2,615",MARMLS,A11678807,Christopher Wands Douglas Elliman Anca Mirescu...,Christopher Wands Douglas Elliman Janet Hoyos ...
2,2025-01-27,9601 Collins Ave Ph 206 in Bal Harbour,"$3,650,000",2,3.5,"2,690",1998,"$1,357",MARMLS,A11566310,Karen Reiter One Sotheby's International Realt...,Joelle Oiknine One Sotheby's International Rea...
3,2025-01-29,9601 Collins Ave #1608 in Bal Harbour,"$3,300,000",2,2.5,"2,250",1998,"$1,467",MARMLS,A11703368,Joelle Oiknine One Sotheby's International Rea...,Joelle Oiknine One Sotheby's International Rea...
4,2025-02-02,17121 Collins Ave #3304 in Sunny Isles Beach,"$2,850,000",3,3.5,"1,933",2009,"$1,474",MARMLS,A11623617,,
5,2025-01-31,848 Brickell Key Dr #4201 in Miami,"$2,800,000",3,2.5,"2,353",2001,"$1,190",MARMLS,A11451983,Daysi Morey Berkshire Hathaway HomeServices Fl...,Maggie Joya BHHS EWM Realty
6,2025-01-31,848 Brickell Key Dr #3701 in Miami,"$2,525,000",3,3.0,"2,353",2001,"$1,073",MARMLS,A11633034,"Al Beteta Solid Group Realty, LLC",Katherina Santana Keller Williams Capital Real...
7,2025-01-31,10201 E Bay Harbor Dr #203 in Bay Harbor Islands,"$2,200,000",3,3.5,"1,660",2016,"$1,325",MARMLS,A11634043,Fernando Rodriguez Fortune Christie's Internat...,"Viktoriya Donisova Ho Compass Florida, LLC."
8,2025-01-30,3400 SW 27th Ave #304 in Miami,"$1,750,000",2,2.5,"1,530",2001,"$1,144",MARMLS,A11651850,"Maite Alvarez Compass Florida, LLC.",Mitzi Mitchell Cervera Real Estate Inc.
9,2025-02-03,650 NE 32nd St #4001 in Miami,"$1,575,000",4,4.5,"1,872",2018,$841,MARMLS,A11664569,Adam Redolfi Alure Capital,Adam Redolfi Alure Capital
